<a href="https://colab.research.google.com/github/santiagovazquezff/circuit_fault_detection/blob/main/demo/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demo for circuit_fault_pipeline**

This demo shows how our trained Random Forest model classifies circuit states (healthy / open / short) from waveform data.

## How to use
1. Click on the blue (`Open in Colab`) button at the top to open the demo in Google Colab (a cloud based Python notebook for ML)
2. In Google Colab run all cells (`Runtime > Run all`) to execute the code
3. As stated in the command window when ran, upload one of the sample CSV files from circuit_fault_detection/demo/ in the GitHub repo (you can upload your own simulated CSV from Altium, consult the lab report)
4. As also stated in the command window, enter input amplitude and frequency for the data on the CSV file (a dictionary for the sample CSV files with the corresponding amplitudes and frequencies is provided below)
5. View predicted class, probabilities, and waveform plot

## Imports and file loading

In [ ]:
import os, sys, json, numpy as np, pandas as pd, joblib
import matplotlib.pyplot as plt
from collections import Counter

file_dictionary = {
  "demo_file_1": {"amplitude": 0.283,
                  "frequency": 200},
  "demo_file_2": {"amplitude": 0.283,
                  "frequency": 200},
  "demo_file_3": {"amplitude": 0.283,
                  "frequency": 200}
  }

!mkdir -p model_artifacts
!wget -q https://raw.githubusercontent.com/santiagovazquezff/circuit_fault_detection/main/model_artifacts/rf_model.joblib -O model_artifacts/rf_model.joblib
!wget -q https://raw.githubusercontent.com/santiagovazquezff/circuit_fault_detection/main/model_artifacts/meta.json     -O model_artifacts/meta.json

rf = joblib.load("model_artifacts/rf_model.joblib")
with open("model_artifacts/meta.json") as f:
    meta = json.load(f)

classes = meta.get("classes", [])
feat_names = meta.get("features", [])
expects_f0 = ("f0" in feat_names)

print("Model loaded")
print("Classes:", classes)
print("Feature order:", feat_names)

## User inputs - **follow instructions in terminal window**

In [ ]:
from google.colab import files
print("""
\n\033[1mInstructions for uploading demo files:\033[0m
\n1. In the GitHub repo santiagovazquezff/circuit_fault_detection/demo, select one of the CSV files included:""")
print("\nFile dictionary:")
print(file_dictionary)
print("""
2. Click on the "download raw file" button to download the file
\n3. Upload the file in the box below
""")
uploaded = files.upload()
csv_files = list(uploaded.keys())
if not csv_files:
    raise SystemExit("No CSVs uploaded.")

vin_pk = float(input("\nEnter input amplitude of waveform uploaded [consult dictionary below step 1 terminal window]: ") or 0.283)
f0     = float(input("\nEnter excitation frequency of waveform uploaded [consult dictionary below step 1 terminal window]: "))

## Model functions and uploaded data processing

In [ ]:
TIME_WINDOW, N_POINTS = 0.1, 16384

def read_df(df):
    assert "s" in df.columns, "CSV must have a 's' time column (seconds)."
    t = df["s"].to_numpy(float)
    V = df.iloc[:, 1:].to_numpy(float)
    if V.ndim == 1:
        V = V[:, None]
    return t, V

def standardise(t, v, time_window=TIME_WINDOW, n_points=N_POINTS):
    t0 = float(t[0])
    mask = (t - t0) < time_window
    t2, v2 = t[mask], v[mask]
    if t2.size < 2:
        raise ValueError("Not enough points inside the time window for interpolation.")
    t_fit = np.linspace(0.0, time_window, n_points, endpoint=False)
    v_fit = np.interp(t_fit, t2 - t0, v2)
    return t_fit, v_fit

def lockin_features(v_fit, t_fit, f0_hz):
    x, tt = v_fit.astype(float), t_fit.astype(float)
    dc = float(np.mean(x)); xz = x - dc; N = x.size
    def ap(freq):
        w = 2*np.pi*freq
        c = np.cos(w*tt); s = np.sin(w*tt)
        a = (2.0/N)*np.dot(xz, c); b = (2.0/N)*np.dot(xz, s)
        A = float(np.hypot(a, b)); ph = float(np.arctan2(-b, a))
        return A, ph
    A1, ph = ap(f0_hz); A2,_ = ap(2*f0_hz); A3,_ = ap(3*f0_hz)
    thd = (np.sqrt(A2**2 + A3**2)/A1) if A1 > 0 else 0.0
    ms_total = float(np.mean(x**2)); ms_dc = dc**2
    ms_tones = (A1**2 + A2**2 + A3**2)/2.0
    noise_rms = float(np.sqrt(max(ms_total - ms_dc - ms_tones, 0.0)))
    return A1, ph, thd, noise_rms, A2, A3

def features_from_df(df, f0_hz, vin_pk_val, include_f0=True):
    t, V = read_df(df)
    rows = []
    for k in range(V.shape[1]):
        t_fit, v_fit = standardise(t, V[:, k])
        A1, ph, thd, noise_rms, A2, A3 = lockin_features(v_fit, t_fit, f0_hz)
        feats = [A1/vin_pk_val, ph, thd, noise_rms/vin_pk_val, A2/vin_pk_val, A3/vin_pk_val]
        if include_f0:
            feats.append(float(f0_hz))
        rows.append(feats)
    return np.array(rows, float), t, V

def predict_with_features(X):
    preds = rf.predict(X)
    proba = rf.predict_proba(X).mean(axis=0)  # average confidence across runs
    maj, cnt = Counter(preds).most_common(1)[0]
    return preds, proba, maj, cnt

for fname in csv_files:
    print("\n" + "-"*80)
    print("File:", fname)
    df = pd.read_csv(fname)

    X_with, t_raw, V_raw = features_from_df(df, f0_hz=f0, vin_pk_val=vin_pk, include_f0=expects_f0)
    preds_w, proba_w, maj_w, cnt_w = predict_with_features(X_with)

## Results for data uploaded - **see terminal window**

In [ ]:
    print(f"\n\n\033[1mFINAL PREDICTION: {maj_w}  ({cnt_w}/{len(preds_w)} runs)\033[0m ")
    print("\nClass probabilities:")
    for cls, p in zip(rf.classes_, proba_w):
        print(f"  {cls}: {p:.3f}")

    run0 = V_raw[:, 0]
    ms_window = 15e-3
    mask_vis = (t_raw - t_raw[0]) < ms_window
    t_vis = (t_raw[mask_vis] - t_raw[0]) * 1e3  # ms
    v_vis = run0[mask_vis]

    print("\nPlot of raw data for user to confirm prediction as either healthy, open, or short:")
    plt.figure(figsize=(6.2, 3.6))
    plt.plot(t_vis, v_vis)
    plt.xlabel("Time [ms]")
    plt.ylabel("Vout [V]")
    plt.title(f"{fname} — waveform (first 15 ms). Predicted (with f0): {maj_w}")
    plt.tight_layout()
    plt.show()